# Visual Geometry Group (VGG16) transfer learning

Tests of the architecture with VGG16 pre-trained model.

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
TRAIN_DIR = '../vgg16/data/train'
seed = 2024
    
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    subset='training',
    seed=seed,
    target_size=(32, 32),
    batch_size=16
)

val_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
    seed=seed,
    target_size=(32, 32),
    batch_size=16
)

Found 72000 images belonging to 10 classes.
Found 18000 images belonging to 10 classes.


In [4]:
vgg16 = keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(32,32,3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [5]:
for layer in vgg16.layers:
    layer.trainable = False

In [10]:
def plot_accuracy_and_loss(history_df, name, idx):
    # Plot and save accuraccy
    plt.plot(history_df['accuracy'])
    plt.plot(history_df['val_accuracy'])
    plt.title(f'{name}: accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    path = f'train_history/{name}/{idx}_accuracy.png'
    plt.savefig(path)
    print(f'Accuracy plot is saved to: {path}')
    plt.close()
    # Plot and save loss
    plt.figure()
    plt.plot(history_df['loss'])
    plt.plot(history_df['val_loss'])
    plt.title(f'{name}: loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    path = f'train_history/{name}/{idx}_loss.png'
    plt.savefig(path)
    print(f'Loss plot is saved to: {path}')
    plt.close()

def plot_confusion_matrix(name, idx):
    model = keras.models.load_model('train_history/'+name+'/'+idx+'.keras')
    X, y = zip(*(val_generator[i] for i in range(len(val_generator))))
    X_test, y_test = np.vstack(X), np.vstack(y)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    y_test = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(0, 10), yticklabels=range(0, 10))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'{name} Confusion Matrix. Accuracy: {accuracy * 100:.2f}%')
    path = f'train_history/{name}/{idx}_confusion_matrix.png'
    plt.savefig(path)
    print(f'Confusion matrix is saved to: {path}')
    plt.close()
    return accuracy

def plot_accuracy_boxplot(accuracy, name): 
    plt.figure()
    plt.boxplot(accuracy)
    plt.xticks([1], [name])
    plt.title(f'{name}: Accuracy boxplot')
    path = f'train_history/{name}/boxplot.png'
    plt.savefig(path)
    print(f'Accuracy boxplot is saved to: {path}')
    plt.close()

## Test 1: VGG16 pre-trained model with extra layers

VGG16 - (Flatten) Dense (50) - Dense (20) - Dense (10)

In [12]:
name = 'vgg1'
n_epochs = 10
n_repeat = 5
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

    

Attempt #1
Epoch 1/10
4500/4500 [==============================] - 699s 154ms/step - loss: 1.9860 - accuracy: 0.2869 - val_loss: 1.5103 - val_accuracy: 0.4458
Epoch 2/10
4500/4500 [==============================] - 670s 149ms/step - loss: 1.5585 - accuracy: 0.4287 - val_loss: 1.4037 - val_accuracy: 0.5019
Epoch 3/10
4500/4500 [==============================] - 674s 150ms/step - loss: 1.5005 - accuracy: 0.4544 - val_loss: 1.3872 - val_accuracy: 0.5179
Epoch 4/10
4500/4500 [==============================] - 693s 154ms/step - loss: 1.4631 - accuracy: 0.4687 - val_loss: 1.3719 - val_accuracy: 0.5264
Epoch 5/10
4500/4500 [==============================] - 704s 156ms/step - loss: 1.4365 - accuracy: 0.4806 - val_loss: 1.3956 - val_accuracy: 0.4949
Epoch 6/10
4500/4500 [==============================] - 720s 160ms/step - loss: 1.4168 - accuracy: 0.4881 - val_loss: 1.3799 - val_accuracy: 0.5250
Epoch 7/10
4500/4500 [==============================] - 702s 156ms/step - loss: 1.3976 - accuracy: 0.

4500/4500 [==============================] - 748s 166ms/step - loss: 1.3786 - accuracy: 0.5014 - val_loss: 1.3815 - val_accuracy: 0.5202
Epoch 9/10
4500/4500 [==============================] - 737s 164ms/step - loss: 1.3627 - accuracy: 0.5070 - val_loss: 1.3933 - val_accuracy: 0.5194
Epoch 10/10
4500/4500 [==============================] - 743s 165ms/step - loss: 1.3518 - accuracy: 0.5119 - val_loss: 1.3789 - val_accuracy: 0.5243
Accuracy plot is saved to: train_history/vgg1/4_accuracy.png
Loss plot is saved to: train_history/vgg1/4_loss.png
563/563 [==============================] - 137s 241ms/step
Accuracy: 52.43%
Confusion matrix is saved to: train_history/vgg1/4_confusion_matrix.png
Attempt accuracy: 52.43%
Attempts accuracy is saved to train_history/vgg1/accuracy.csv
Accuracy boxplot is saved to: train_history/vgg1/boxplot.png


## Test 2: VGG1 with dropout layers, rescaling factor and 5 epochs

VGG16 - (Flatten) Dense (50) - Dropout - Dense (20) - Dropout - Dense (10)

In [32]:
datagen = ImageDataGenerator(validation_split=0.2,
                             rescale=1./255)

train_generator = datagen.flow_from_directory(
    TRAIN_DIR, 
    subset='training',
    seed=seed,
    target_size=(32, 32),
    batch_size=16
)

val_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
    seed=seed,
    target_size=(32, 32),
    batch_size=16
)

Found 72000 images belonging to 10 classes.
Found 18000 images belonging to 10 classes.


In [33]:
name = 'vgg2'
n_epochs = 5
n_repeat = 5
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)


Attempt #1
Epoch 1/5
4500/4500 [==============================] - 814s 180ms/step - loss: 2.0188 - accuracy: 0.2464 - val_loss: 1.6946 - val_accuracy: 0.3977
Epoch 2/5
4500/4500 [==============================] - 868s 193ms/step - loss: 1.8820 - accuracy: 0.2965 - val_loss: 1.6698 - val_accuracy: 0.3845
Epoch 3/5
4500/4500 [==============================] - 2361s 525ms/step - loss: 1.8520 - accuracy: 0.3075 - val_loss: 1.6259 - val_accuracy: 0.4082
Epoch 4/5
4500/4500 [==============================] - 916s 203ms/step - loss: 1.8382 - accuracy: 0.3150 - val_loss: 1.6114 - val_accuracy: 0.4378
Epoch 5/5
4500/4500 [==============================] - 883s 196ms/step - loss: 1.8290 - accuracy: 0.3198 - val_loss: 1.5961 - val_accuracy: 0.4416
Accuracy plot is saved to: train_history/vgg2/0_accuracy.png
Loss plot is saved to: train_history/vgg2/0_loss.png
563/563 [==============================] - 152s 269ms/step
Accuracy: 44.16%
Confusion matrix is saved to: train_history/vgg2/0_confusion_ma

## Test 3: VGG1 with normalization, rescaling factor and 5 epochs

VGG16 - (Flatten) (Batch normalization) Dense (50) - Dense (20) - Dense (10)

In [38]:
name = 'vgg3'
n_epochs = 5
n_repeat = 5
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/5
4500/4500 [==============================] - 1597s 353ms/step - loss: 1.6800 - accuracy: 0.3942 - val_loss: 1.4067 - val_accuracy: 0.5072
Epoch 2/5
4500/4500 [==============================] - 1277s 284ms/step - loss: 1.5594 - accuracy: 0.4388 - val_loss: 1.3886 - val_accuracy: 0.5197
Epoch 3/5
4500/4500 [==============================] - 1063s 236ms/step - loss: 1.5235 - accuracy: 0.4521 - val_loss: 1.3849 - val_accuracy: 0.5193
Epoch 4/5
4500/4500 [==============================] - 913s 203ms/step - loss: 1.4998 - accuracy: 0.4595 - val_loss: 1.3602 - val_accuracy: 0.5307
Epoch 5/5
4500/4500 [==============================] - 789s 175ms/step - loss: 1.4819 - accuracy: 0.4680 - val_loss: 1.3746 - val_accuracy: 0.5252
Accuracy plot is saved to: train_history/vgg3/0_accuracy.png
Loss plot is saved to: train_history/vgg3/0_loss.png
563/563 [==============================] - 146s 258ms/step
Accuracy: 52.52%
Confusion matrix is saved to: train_history/vgg3/0_confusion_

## Test 3: VGG3 with dropout layers

VGG16 - (Flatten) (Batch normalization) Dense (50) - Dropout - Dense (20) - Dropout - Dense (10)

In [40]:
name = 'vgg4'
n_epochs = 5
n_repeat = 5
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/5
4500/4500 [==============================] - 1146s 254ms/step - loss: 2.0543 - accuracy: 0.2439 - val_loss: 1.6866 - val_accuracy: 0.4050
Epoch 2/5
4500/4500 [==============================] - 760s 169ms/step - loss: 1.8928 - accuracy: 0.2997 - val_loss: 1.6181 - val_accuracy: 0.4471
Epoch 3/5
4500/4500 [==============================] - 777s 173ms/step - loss: 1.8533 - accuracy: 0.3201 - val_loss: 1.5870 - val_accuracy: 0.4549
Epoch 4/5
4500/4500 [==============================] - 776s 172ms/step - loss: 1.8350 - accuracy: 0.3257 - val_loss: 1.5400 - val_accuracy: 0.4646
Epoch 5/5
4500/4500 [==============================] - 764s 170ms/step - loss: 1.8183 - accuracy: 0.3362 - val_loss: 1.5474 - val_accuracy: 0.4599
Accuracy plot is saved to: train_history/vgg4/0_accuracy.png
Loss plot is saved to: train_history/vgg4/0_loss.png
563/563 [==============================] - 127s 224ms/step
Accuracy: 45.99%
Confusion matrix is saved to: train_history/vgg4/0_confusion_ma

## Test 5: VGG3 with adamax optimizer and 25 epochs (without repeating)

VGG16 - (Flatten) (Batch normalization) Dense (50) - Dense (20) - Dense (10)

In [41]:
name = 'vgg5'
n_epochs = 25
n_repeat = 1
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/25
4500/4500 [==============================] - 739s 164ms/step - loss: 1.7410 - accuracy: 0.3727 - val_loss: 1.4384 - val_accuracy: 0.4913
Epoch 2/25
4500/4500 [==============================] - 739s 164ms/step - loss: 1.5852 - accuracy: 0.4304 - val_loss: 1.3916 - val_accuracy: 0.5128
Epoch 3/25
4500/4500 [==============================] - 3763s 836ms/step - loss: 1.5436 - accuracy: 0.4461 - val_loss: 1.3578 - val_accuracy: 0.5274
Epoch 4/25
4500/4500 [==============================] - 904s 201ms/step - loss: 1.5175 - accuracy: 0.4549 - val_loss: 1.3420 - val_accuracy: 0.5316
Epoch 5/25
4500/4500 [==============================] - 911s 203ms/step - loss: 1.4946 - accuracy: 0.4633 - val_loss: 1.3393 - val_accuracy: 0.5300
Epoch 6/25
4500/4500 [==============================] - 814s 181ms/step - loss: 1.4827 - accuracy: 0.4679 - val_loss: 1.3314 - val_accuracy: 0.5381
Epoch 7/25
4500/4500 [==============================] - 720s 160ms/step - loss: 1.4699 - accuracy: 0

# Failed model with data augmentation (come back later)

In [13]:
class MixupImageDataGenerator():
    def __init__(self, generator, directory, batch_size, img_height, img_width, alpha=0.2, subset=None):
        """Constructor for mixup image data generator.
        Arguments:
            generator {object} -- An instance of Keras ImageDataGenerator.
            directory {str} -- Image directory.
            batch_size {int} -- Batch size.
            img_height {int} -- Image height in pixels.
            img_width {int} -- Image width in pixels.
        Keyword Arguments:
            alpha {float} -- Mixup beta distribution alpha parameter. (default: {0.2})
            subset {str} -- 'training' or 'validation' if validation_split is specified in
            `generator` (ImageDataGenerator).(default: {None})
        """

        self.batch_index = 0
        self.batch_size = batch_size
        self.alpha = alpha

        # First iterator yielding tuples of (x, y)
        self.generator1 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset)

        # Second iterator yielding tuples of (x, y)
        self.generator2 = generator.flow_from_directory(directory,
                                                        target_size=(
                                                            img_height, img_width),
                                                        class_mode="categorical",
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        subset=subset)

        # Number of images across all classes in image directory.
        self.n = self.generator1.samples

    def reset_index(self):
        """Reset the generator indexes array.
        """

        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def on_epoch_end(self):
        self.reset_index()

    def reset(self):
        self.batch_index = 0

    def __len__(self):
        # round up
        return (self.n + self.batch_size - 1) // self.batch_size

    def get_steps_per_epoch(self):
        """Get number of steps per epoch based on batch size and
        number of images.
        Returns:
            int -- steps per epoch.
        """

        return self.n // self.batch_size

    def __next__(self):
        """Get next batch input/output pair.
        Returns:
            tuple -- batch of input/output pair, (inputs, outputs).
        """

        if self.batch_index == 0:
            self.reset_index()

        current_index = (self.batch_index * self.batch_size) % self.n
        if self.n > current_index + self.batch_size:
            self.batch_index += 1
        else:
            self.batch_index = 0

        # random sample the lambda value from beta distribution.
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)

        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        # Get a pair of inputs and outputs from two iterators.
        X1, y1 = self.generator1.next()
        X2, y2 = self.generator2.next()

        # Perform the mixup.
        X = X1 * X_l + X2 * (1 - X_l)
        y = y1 * y_l + y2 * (1 - y_l)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)

In [15]:
batch_size=16
img_height=32

training_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = MixupImageDataGenerator(generator=training_datagen,
                                          directory=TRAIN_DIR,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_height,
                                          subset='training')

val_generator = training_datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
    seed=seed,
    target_size=(32, 32),
    batch_size=batch_size
)

Found 72000 images belonging to 10 classes.
Found 72000 images belonging to 10 classes.
Found 18000 images belonging to 10 classes.


In [17]:
name = 'vgg2'
n_epochs = 10
n_repeat = 5
keras.utils.set_random_seed(seed)
accuracy = []

for idx in range(n_repeat):
    print(f'Attempt #{idx + 1}')
    x = keras.layers.Flatten()(vgg16.output)
    x = keras.layers.Dense(units=50, activation='relu')(x)
    x = keras.layers.Dense(units=20, activation='relu')(x)
    x = keras.layers.Dense(units=10, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=vgg16.input, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=n_epochs
    )
    
    model.save(f'train_history/{name}/{idx}.keras')
    history_df = pd.DataFrame(history.history) 
    hist_csv_file = 'train_history/' + name + '/' + str(idx) + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        history_df.to_csv(f)
    plot_accuracy_and_loss(history_df, name, str(idx))
    curr_accuracy = plot_confusion_matrix(name, str(idx))
    accuracy.append(curr_accuracy)
    print(f'Attempt accuracy: {curr_accuracy * 100:.2f}%')
    
accuracy_df = pd.DataFrame(data=accuracy, columns = ['accuracy'])
accuracy_csv_file = 'train_history/' + name + '/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')
plot_accuracy_boxplot(accuracy, name)

Attempt #1
Epoch 1/10
  13443/Unknown - 1950s 145ms/step - loss: 1.7729 - accuracy: 0.3861

KeyboardInterrupt: 